# Radial distribution functions

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mdtraj

In [ ]:
dir = '../c2mim_bf4/'
trj = mdtraj.load(dir+'traj.dcd', top=dir+'config.pdb')
print(trj)

In [ ]:
atoms, bonds = trj.topology.to_dataframe()
atoms.head()

In [ ]:
res = pd.unique(atoms['resName'])
res

In [ ]:
mols = pd.DataFrame(index=res, columns=['natoms', 'nmols'])
for spec in res:
    #mols[spec] = trj.topology.select(f"resname {spec}")
    moli = atoms.query(f"resName == '{spec}'").index
    i = moli[0]
    j = moli[-1]
    nmols = atoms.iloc[j].resSeq - atoms.iloc[i].resSeq + 1
    natoms = int((j - i + 1) / nmols)
    mols.loc[spec] = [natoms, nmols]
print(mols)

In [ ]:
def showatoms(atoms, mols, res):
    offset = atoms.query(f"resName == '{res}'").index[0]
    print(atoms[offset:offset+mols.loc[res].natoms])

In [ ]:
showatoms(atoms, mols, 'c2c')

In [ ]:
showatoms(atoms, mols, 'BF4')

In [ ]:
top = trj.topology
ati = top.select("resname c2c and (name == N1 or name == N2)")
atj = top.select("resname BF4 and element == B")
pairs = [ [i, j] for i in ati for j in atj ]

In [ ]:
len(ati), len(atj), len(pairs)

In [ ]:
rdf = mdtraj.compute_rdf(trj, pairs, bin_width=0.01, r_range=(0.0, 2.0))

In [ ]:
fig, ax = plt.subplots()
ax.plot((0, 2.0), (1, 1), 'k--')
ax.plot(rdf[0], rdf[1], label='N - B')
ax.set_xlabel('r (nm)')
ax.set_ylabel('g(r)')
#ax.set_ylim(0, 5)
ax.legend()
plt.show()